In [1]:
import pandas as pd
import numpy as np
from joblib import dump, load
import time
import datetime
import os

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

import tensorflow as tf
import tensorflow.keras as tfk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras import callbacks

# import kerastuner as kt
# from kerastuner.tuners import RandomSearch

from matplotlib import pyplot as plt

In [2]:
%load_ext tensorboard

### Some useful functions

In [3]:
def get_confusion_matrix(true_pos, false_pos, true_neg, false_neg):
    conf_matrix = np.array([
                            [true_pos, false_pos],
                            [false_neg, true_neg]
                           ])
    
    return pd.DataFrame(conf_matrix, columns=['1', '0'], index=['1', '0'])

In [4]:
def learning_plot(model, metric):
    
    fig = plt.figure()
    
    fig = plt.plot(model.history[metric], color='black')
    fig = plt.plot(model.history['val_'+metric], color='blue')

    plt.title('Changes in {} over training run'.format(metric))
    plt.xlabel('Epoch')
    plt.ylabel(metric)
    
    plt.legend(['train', 'val'], loc='upper right')
    
    return fig

In [5]:
def learning_recall(model, positives_flag=True):
    
    if positives_flag:
        recall = [tp / (tp+fn) for tp, fn in zip(model.history['true_positives'], model.history['false_negatives'])]
        val_recall = [tp / (tp+fn) for tp, fn in zip(model.history['val_true_positives'], model.history['val_false_negatives'])]
        recall_type = 'positive'
    else:
        recall = [tn / (tn+fp) for tn, fp in zip(model.history['true_negatives'], model.history['false_positives'])]
        val_recall = [tn / (tn+fp) for tn, fp in zip(model.history['val_true_negatives'], model.history['val_false_positives'])]
        recall_type = 'negative'
        
    fig = plt.figure()
    
    fig = plt.plot(recall, color='black')
    fig = plt.plot(val_recall, color='blue')
    
    plt.title('Changes in {} recall rate over training run.'.format(recall_type))
    plt.xlabel('Epoch')
    plt.ylabel('{} recall rate'.format(recall_type))
    
    return fig

In [6]:
def random_search_models(num_models, input_dims, compile_metrics, seed):
    
    np.random.seed(seed)
    
    all_models = []
    
    for m in range(num_models): 
        
        num_hidden_layers = np.random.randint(1, 11)
        all_models.append(build_model(num_hidden_layers, input_dims, compile_metrics))
    
    return all_models

def build_layers(num_layers, input_dims):

    layers = []
    
    for i in range(num_layers):
        
        num_units = np.random.randint(2, 27)
        reg_val = 10**(-4*np.random.rand())
        if i==0:
            layers.append(Dense(
                                units = num_units,
                                input_dim = input_dims,
                                activation = 'relu',
                                kernel_regularizer = l2(l2=reg_val)
                               ))
        else:
            layers.append(Dense(
                                units = num_units,
                                activation = 'relu',
                                kernel_regularizer = l2(l2=reg_val)
                               ))
    reg_val = 10**(-4*np.random.rand())
    layers.append(Dense(
                        units = 1,
                        activation = 'sigmoid',
                        kernel_regularizer = l2(l2=reg_val)
                       ))
    return layers

def build_model(num_hidden_layers, input_dims, compile_metrics):

    model = Sequential()
    
    layers = build_layers(num_hidden_layers, input_dims)
    for l in layers:
        model.add(l)
    
    learning_rate = 10**(-4*np.random.rand())
    model.compile(optimizer = Adam(learning_rate), loss='binary_crossentropy', metrics=[compile_metrics])
    
    return model

### Load and prepare data

In [7]:
df = pd.read_csv('jazz.csv', sep='|')

In [8]:
# Define features to analyze
features = [
            'danceability',
            'energy',
            'speechiness',
            'acousticness',
            'instrumentalness',
            'liveness',
            'valence',
            'num_samples',
            'end_of_fade_in',
            'loudness',
            'tempo',
            'key',
            'mode',
            'bars_num',
            'bars_duration_var',
            'beats_duration_var',
            'sections_num',
            'sections_duration_mean',
            'sections_duration_var',
            'loudness_var',
            'tempo_var',
            'key_var',
            'mode_var',
            'segments_duration_var',
            'segments_duration_mean',
            'pitches_mean',
            'pitches_var',
            'timbre_mean',
            'timbre_var',
            'tatums_duration_var'
           ]

df = df[features+['label']]

# Shuffle data to ungroup class rows
df = df.sample(frac=1, random_state=12).reset_index(drop=True)

In [9]:
df[df['label']==1].shape

(1560, 31)

Split dataset into training, validation, and test. 60/20/20 gives ~936 positive training samples, and ~312 each of positive validation and test samples. Quite small, but hopefully big enough to be meaningful.

In [10]:
X = df[features].copy()
Y = df['label'].copy()

X_train, X_test, y_train, y_test = train_test_split(
                                                    X,
                                                    Y, 
                                                    stratify = Y,
                                                    test_size = 0.2,
                                                    random_state = 42
                                                   )

# Start building model

In [11]:
# First fit how to scale data for the model
pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('imputer', SimpleImputer(strategy='median'))])

scale_model = pipeline.fit(X_train)
X_train = scale_model.transform(X_train)
X_test = scale_model.transform(X_test)

# Save for later use
dump(scale_model, 'scaler.joblib') 

['scaler.joblib']

In [12]:
logdir = 'logs/scalars/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

metrics = [
           tfk.metrics.Precision(),
           tfk.metrics.TruePositives(),
           tfk.metrics.TrueNegatives(),
           tfk.metrics.FalsePositives(),
           tfk.metrics.FalseNegatives(),
           tfk.metrics.AUC(curve='PR')
          ]

models = random_search_models(num_models=25, input_dims=len(features), compile_metrics=metrics, seed=42)

for i, model in enumerate(models):

    training_history = model.fit(
                                 X_train,
                                 y_train,
                                 batch_size = 128,
                                 verbose = 0,
                                 epochs = 500,
                                 validation_split = 0.2,
                                 callbacks=[callbacks.TensorBoard(log_dir=logdir+'-'+str(i))],
                                )
    
    print('Average validation loss for model ', str(i+1), ': ', np.average(training_history.history['val_loss']))

Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Average validation loss for model  1 :  1.247082142829895
Average validation loss for model  2 :  0.7802540791034699
Average validation loss for model  3 :  0.7538687134981156
Average validation loss for model  4 :  0.711441454410553
Average validation loss for model  5 :  0.6908894851207733
Average validation loss for model  6 :  0.7697943080663681
Average validation loss for model  7 :  1.1231872308254243
Average validation loss for model  8 :  0.549254743039608
Average validation loss for model  9 :  0.6144829293489457
Average validation loss for model  10 :  0.6871076166629791
Average validation loss for model  11 :  0.5356809813976288
Average validation loss for model  12 :  0.7370461231470108
Average validation loss for model  13 :  0.9959176428318024
Average validation loss for model  14 :  0.5383603849411011
Average validation loss for model  15 :  0.8163906596899032
Average validation loss for model  16 : 

In [13]:
%tensorboard --logdir logs/scalars --host localhost

Reusing TensorBoard on port 6006 (pid 252), started 0:15:12 ago. (Use '!kill 252' to kill it.)

In [14]:
# TODO: find a dynamic naming schedule that's not too long (datetimes make the filename too long for Windows)
# kt_dir = os.path.normpath('./logs/005')

In [15]:
# tuner = RandomSearch(
#                      build_model,
#                      objective = kt.Objective('val_loss', direction='min'),
#                      max_trials = 25,
#                      executions_per_trial = 3,
#                      directory = os.path.normpath(kt_dir),
#                      project_name = 'jazzy-curator'
#                     )

# tuner.search(
#               X_train, 
#               y_train,
#               epochs = 500,
#               validation_split = 0.2,
#               batch_size = 128,
#               callbacks = [callbacks.TensorBoard(kt_dir)],
#               shuffle = True
#              )

In [16]:
# models = tuner.get_best_models(num_models=5)

In [17]:
# tuner.oracle.get_best_trials()[0].trial_id

In [18]:
# kt_dir

In [19]:
# %tensorboard --logdir=logs/

# Evaluate model on validation set

In [20]:
# learning_plot(models[0], 'loss')

In [21]:
# learning_plot(jazz_model, 'auc')

In [22]:
# learning_plot(jazz_model, 'precision')

In [23]:
# learning_recall(jazz_model, True)

In [24]:
# learning_recall(jazz_model, False)

In [25]:
# loss = jazz_model.history['loss'][-1]
# val_loss = jazz_model.history['val_loss'][-1]
# print('Loss for training set is {}, while loss for validation set is {}. This gives a difference of {}'\
#       .format(
#               round(loss,4),
#               round(val_loss,4), 
#               round(val_loss-loss, 4)))

In [26]:
# auc = jazz_model.history['auc'][-1]
# val_auc = jazz_model.history['val_auc'][-1]
# print('AUC for training set is {}, while AUC for validation set is {}. This gives a difference of {}'\
#       .format(round(auc, 2), round(val_auc, 2), round(auc-val_auc,2)))

In [27]:
# true_positives_val = jazz_model.history['val_true_positives'][-1]
# false_positives_val = jazz_model.history['val_false_positives'][-1]
# true_negatives_val = jazz_model.history['val_true_negatives'][-1]
# false_negatives_val = jazz_model.history['val_false_negatives'][-1]

# val_conf_matrix = get_confusion_matrix(
#                                        true_positives_val,
#                                        false_positives_val,
#                                        true_negatives_val,
#                                        false_negatives_val
#                                        )

# print('Confusion_matrix:\n{}'.format(val_conf_matrix))

In [28]:
# print('True recall rate is {}'\
#       .format(round(val_conf_matrix.loc['1', '1']/(val_conf_matrix.loc['1', '1']+val_conf_matrix.loc['0', '1']),2)))
# print('Precision is {}'.format(jazz_model.history['val_precision'][-1]))

Looks very promising on validation set! Let's check test set

In [29]:
# loss_test, precision_test, true_positives_test, true_negatives_test, false_positives_test, false_negatives_test, auc_test = model.evaluate(X_test, y_test)

In [30]:
# print('AUC for test set is {}.'.format(round(auc_test,2)))

In [31]:
# confusion_matrix = get_confusion_matrix(
#                                         true_positives_test,
#                                         false_positives_test,
#                                         true_negatives_test,
#                                         false_negatives_test
#                                        )

In [32]:
# print(confusion_matrix)

In [33]:
# print(
#       'True recall rate is {}'\
#       .format(round(
#                     confusion_matrix.loc['1', '1']/
#                     (confusion_matrix.loc['1', '1']+confusion_matrix.loc['0', '1']),
#                     2
#                    )
#              )
#      )
# print('Precision is {}'.format(round(precision_test,2)))

# Save model

In [34]:
# model.save('jazz_model')